# Integer Programming

In [15]:
import pyomo.environ as pyo

from itertools import product

users = ['u0', 'u1']
products = ['p0', 'p1', 'p2']

ups = list(product(users, products))

costs = {(u, p): 4  if p == 'p0' else 6 for u, p in ups}
profits = {(u, p): 6 if u == 'u0' else 4 for u, p in ups}

budget = 20
max_assignment = 2
qs = {p: 1 for p in products}

In [16]:
ups

[('u0', 'p0'),
 ('u0', 'p1'),
 ('u0', 'p2'),
 ('u1', 'p0'),
 ('u1', 'p1'),
 ('u1', 'p2')]

In [19]:
model = pyo.ConcreteModel('recommendation')
model.x = pyo.Var(users, products, domain=pyo.Binary)

# budget constraint
def budget_rule(mdl):
    return sum(costs[u, p] * mdl.x[u, p] for u, p in ups) <= budget
model.budget = pyo.Constraint(rule=budget_rule)

# assignment_rule:
def assignment_rule(mdl, u):
    return sum(mdl.x[u, p] for p in products) <= max_assignment
model.assignment = pyo.Constraint(users, rule=assignment_rule)

# balance rule. We want at least qs[p] person to view the product p.
def balance_rule(mdl, p):
    return sum(profits[u, p] * mdl.x[u, p] for u in users) >= qs[p]
model.balance = pyo.Constraint(products, rule=balance_rule)

# Objective function
def objective_func(mdl):
    return sum(profits[u, p] * mdl.x[u, p] for u, p in ups)
model.objective = pyo.Objective(rule=objective_func, sense=pyo.maximize)

In [20]:
solver = pyo.SolverFactory("glpk")
res = solver.solve(model)

pyo.assert_optimal_termination(res)

model.x.display()


x : Size=6, Index=x_index
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('u0', 'p0') :     0 :   1.0 :     1 : False : False : Binary
    ('u0', 'p1') :     0 :   1.0 :     1 : False : False : Binary
    ('u0', 'p2') :     0 :   0.0 :     1 : False : False : Binary
    ('u1', 'p0') :     0 :   1.0 :     1 : False : False : Binary
    ('u1', 'p1') :     0 :   0.0 :     1 : False : False : Binary
    ('u1', 'p2') :     0 :   1.0 :     1 : False : False : Binary
